In [58]:
import pandas as pd
import pyarrow as pa
import numpy as np

import sentencepiece as spm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import torch.nn.functional as F
from tqdm import tqdm
import io
import pickle
from torch.utils.data import DataLoader, Dataset
import os

In [3]:
# Load Parquet data
data = pd.read_parquet('/root/MLXWeek2_copy/hacker_news_data.parquet')
# Now, you can use the `data' DataFrame to analyse and manipulate the data.
print(data.head())

       id                                              title  score   type
0  187635                     ReactOS, a clone of Windows XP      1  story
1  187637  Music Operates Directly On Your Abstract Synta...      1  story
2  187788   Nudge: Helping you make rational daily decisions      1  story
3  187813           Scaling a Microblogging Service - Part I     14  story
4  188029        Good analysis of Powerset by Danny Sullivan      2  story


In [4]:
print(data.columns)

Index(['id', 'title', 'score', 'type'], dtype='object')


In [5]:
# Extract the columns with the correct rows
story_data = data[data["type"] == "story"]

# Extract the relevant columns
relevant_data = story_data[["title", "score"]]

# Remove any null entries
processed_data = relevant_data.dropna()

print(processed_data.head())

                                               title  score
0                     ReactOS, a clone of Windows XP      1
1  Music Operates Directly On Your Abstract Synta...      1
2   Nudge: Helping you make rational daily decisions      1
3           Scaling a Microblogging Service - Part I     14
4        Good analysis of Powerset by Danny Sullivan      2


In [6]:
# Convert to array
data_array = processed_data.to_numpy()
print(data_array)

[['ReactOS, a clone of Windows XP' 1]
 ['Music Operates Directly On Your Abstract Syntax Tree' 1]
 ['Nudge: Helping you make rational daily decisions' 1]
 ...
 ['How to write great Javascript Unit Tests' 6]
 ['Discount Climbing Gear' 1]
 ['Feature Request: Google please unify your Gtalk gadget!' 1]]


In [7]:
# Get the titles
title_array = data_array[:, 0]

In [43]:
# Load the trained SentencePiece model
sp = spm.SentencePieceProcessor()
sp.load('/root/MLXWeek2_copy/weights/skipgram_TechCrunch/techcrunch_sp.model')

# Tokenize titles
tokenized_titles = [sp.encode_as_pieces(title) for title in title_array]

# Turn the tokens into tokenids
id_titles = [[sp.piece_to_id(token) for token in title_tokens] for title_tokens in tokenized_titles]

In [44]:
# A `Dataset` class for generating pairs of id-tokenised titles and scores one at a time, to use in the `DataLoader` to generate mini-batches
class TitlesAndScores(Dataset):
    def __init__(self, titles, scores):
        """
        Args:
            token_ids (list of list of words): Nested list where each sublist contains token IDs for a sentence.
            scores (list of float): List of scores associated with each list of token IDs.
        """
        self.titles = titles
        self.tokenized_titles = [sp.encode_as_pieces(title) for title in self.titles]
        self.id_titles = [[sp.piece_to_id(token) for token in title_tokens] for title_tokens in self.tokenized_titles]
        self.scores = scores
    
    def __len__(self):
        return len(self.scores)
    
    def __getitem__(self, idx):
        return self.id_titles[idx], self.scores[idx]

In [10]:
# A padding function to make sure that every list of title token ids in a mini-batch is the same length, so that the batch can be tensorised
def pad_collate(batch):
    (id_titles, scores) = zip(*batch)
    
    # Padding the sequences with 0
    padded_titles = pad_sequence([torch.tensor(title_ids) for title_ids in id_titles], batch_first=True, padding_value=0)
    
    # Convert scores to a tensor
    tensor_scores = torch.tensor(scores, dtype=torch.float)

    return padded_titles, tensor_scores

In [11]:
# Create the dataset on our hackernews data
sp_dataset = TitlesAndScores(data_array[:,0], data_array[:,1])

# Create the dataLoader on our hackernews data
sp_dataloader = DataLoader(sp_dataset, batch_size=32, shuffle=True, collate_fn=pad_collate)

In [108]:
# Hyperparameters
hidden_dim_1= 64
hidden_dim_2 = 32
num_epochs = 100
vocab_size = 4000
embedding_dim = 300
learning_rate = 0.001
EMBED_MAX_NORM = 1

In [86]:
# Define the SkipGramModel class
class SkipGram_Model(nn.Module):
    def __init__(self, vocab_size, embedding_dim, max_norm=1):
        super(SkipGram_Model, self).__init__()
        self.vocab_size = 4000
        self.embedding_dim = embedding_dim
        self.embeddings = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim, max_norm=max_norm)
        self.linear = nn.Linear(in_features=embedding_dim, out_features=vocab_size)
        
    def forward(self, inputs_):
        x = self.embeddings(inputs_)
        x = self.linear(x)
        return x
    
    def embed(self, inputs_):
        # This method will return just the embeddings
        return self.embeddings(inputs_)



In [47]:
class CustomUnpickler(pickle.Unpickler):
    def find_class(self, module, name):
        if module == "utils":
            return super().find_class(__name__, name)
        return super().find_class(module, name)


In [87]:
skipgram_model = SkipGram_Model(vocab_size, embedding_dim)
skipgram_model = SkipGram_Model(vocab_size, embedding_dim)
print(f"Embedding dimension: {skipgram_model.embedding_dim}")

Embedding dimension: 300


In [88]:
for inputs, targets in sp_dataloader:
    print(f'Max input index: {inputs.max().item()}')  # Add this line to check the maximum index
    inputs = inputs.long()

Max input index: 3973
Max input index: 3844
Max input index: 3955
Max input index: 3926
Max input index: 3982
Max input index: 3985
Max input index: 3884
Max input index: 3955
Max input index: 3834
Max input index: 3845
Max input index: 3882
Max input index: 3955
Max input index: 3982
Max input index: 3879
Max input index: 3987
Max input index: 3963
Max input index: 3882
Max input index: 3975
Max input index: 3897
Max input index: 3982
Max input index: 3964
Max input index: 3854
Max input index: 3815
Max input index: 3975
Max input index: 3882
Max input index: 3987
Max input index: 3898
Max input index: 3987
Max input index: 3987
Max input index: 3985
Max input index: 3964
Max input index: 3757
Max input index: 3818
Max input index: 3902
Max input index: 3941
Max input index: 3846
Max input index: 3859
Max input index: 3958
Max input index: 3876
Max input index: 3762
Max input index: 3915
Max input index: 3735
Max input index: 3987
Max input index: 3963
Max input index: 3975
Max input 

Max input index: 3664
Max input index: 3987
Max input index: 3979
Max input index: 3888
Max input index: 3915
Max input index: 3898
Max input index: 3985
Max input index: 3981
Max input index: 3926
Max input index: 3898
Max input index: 3985
Max input index: 3975
Max input index: 3947
Max input index: 3986
Max input index: 3915
Max input index: 3876
Max input index: 3973
Max input index: 3909
Max input index: 3875
Max input index: 3957
Max input index: 3842
Max input index: 3987
Max input index: 3881
Max input index: 3975
Max input index: 3951
Max input index: 3807
Max input index: 3893
Max input index: 3987
Max input index: 3868
Max input index: 3985
Max input index: 3987
Max input index: 3853
Max input index: 3959
Max input index: 3968
Max input index: 3973
Max input index: 3868
Max input index: 3762
Max input index: 3887
Max input index: 3926
Max input index: 3712
Max input index: 3975
Max input index: 3987
Max input index: 3847
Max input index: 3868
Max input index: 3987
Max input 

In [89]:

# Load the pre-trained SkipGram model
pretrained_model_path = '/root/MLXWeek2_copy/weights/skipgram_TechCrunch/model.pt'
skipgram_model = torch.load(pretrained_model_path, map_location=torch.device('cpu'))
skipgram_model.eval()  # Assuming you're doing inference

# Manually set the embedding_dim if it's not part of the loaded state_dict
skipgram_model.embedding_dim = 300

# Manually add the embed method to the loaded model if it's missing
def embed(self, inputs_):
    return self.embeddings(inputs_)

# Assign the method to the instance
skipgram_model.embed = embed.__get__(skipgram_model, SkipGram_Model)

# Freeze the parameters (weights) of the embedding model
for param in skipgram_model.parameters():
    param.requires_grad = False

# Now skipgram_model has an embedding_dim attribute you can access
print(f"Embedding dimension: {skipgram_model.embedding_dim}")

Embedding dimension: 300


In [135]:
# A model for predicting scores given titles. 
# This particular architecture is intended to use one of the word2vec models we trained earlier (CBOW or Skip-gram) -- with its weights frozen -- to embed the input title tokens.
# It then uses a classic neural network architecture to predict the scores given the embeddings.
# This architecture has two hidden layers with ReLU activations, and a final output layer with no activation (a linear output layer) because we are doing a regression task. 
class ScorePredictor(nn.Module):
    def __init__(self, hidden_dim_1, hidden_dim_2, embedding_model):
        super(ScorePredictor, self).__init__()
        self.embedding_model = embedding_model
        
        # Use embedding_model's embedding dimension
        embedding_dim = self.embedding_model.embedding_dim
        
        self.hidden_1 = nn.Linear(embedding_dim, hidden_dim_1)
        self.hidden_2 = nn.Linear(hidden_dim_1, hidden_dim_2)
        self.output = nn.Linear(hidden_dim_2, 1)
        self.relu = nn.ReLU()

    def forward(self, titles):
        embeddings = self.embedding_model.embed(titles)
        pooled_embeddings = embeddings.mean(dim=1)
        hidden_1_output = self.relu(self.hidden_1(pooled_embeddings))
        hidden_2_output = self.relu(self.hidden_2(hidden_1_output))
        score_predictions = self.output(hidden_2_output)
        return score_predictions

In [117]:
# Model instantiation
sp_model = ScorePredictor(hidden_dim_1, hidden_dim_2, skipgram_model)

# Loss function
sp_loss_function = nn.MSELoss()  # Mean Squared Error Loss for regression

# Optimizer - Excluding the frozen embeddings from being updated
sp_optimizer = optim.Adam(filter(lambda p: p.requires_grad, sp_model.parameters()), lr=learning_rate)

# Training Loop
for epoch in range(num_epochs):
    total_loss = 0
    for inputs, targets in tqdm(sp_dataloader, desc=f'Epoch {epoch+1}/{num_epochs}'):
        
        inputs = inputs.long()
        sp_model.zero_grad()  # Reset gradients
        outputs = sp_model(inputs)  # Forward pass
        loss = sp_loss_function(outputs, targets.view(outputs.size()))  # Compute loss
        loss.backward()  # Backpropagation
        sp_optimizer.step()  # Update the parameters
        total_loss += loss.item()

    # Print the average loss for the epoch
    avg_loss = total_loss / len(sp_dataloader)
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss}')
    
    # Save the model state after each epoch, excluding the frozen embeddings
    torch.save({k: v for k, v in sp_model.state_dict().items() if k != 'embedding_model.embeddings.weight'}, os.path.join("sp_skipgram_weights", f"sp_skipgram_epoch_{epoch+1}.pth"))

    torch.save({
    'score_predictor_state_dict': sp_model.state_dict(),
    'embedding_model_state_dict': skipgram_model.state_dict()
}, '/root/MLXWeek2_copy/combined_model.pth')
    

Epoch 1/100: 100%|██████████| 5170/5170 [00:17<00:00, 292.65it/s]


Epoch 1/100, Loss: 269.09864696524824


Epoch 2/100: 100%|██████████| 5170/5170 [00:17<00:00, 292.64it/s]


Epoch 2/100, Loss: 265.714672713953


Epoch 3/100: 100%|██████████| 5170/5170 [00:17<00:00, 291.73it/s]


...

Epoch 98/100, Loss: 240.40559251386838


Epoch 99/100: 100%|██████████| 5170/5170 [00:18<00:00, 286.07it/s]


Epoch 99/100, Loss: 236.85643698048546


Epoch 100/100: 100%|██████████| 5170/5170 [00:20<00:00, 254.04it/s]


Epoch 100/100, Loss: 237.4271455178178


In [128]:
pretrained_model_path = '/root/MLXWeek2_copy/weights/skipgram_TechCrunch/model.pt'
embedding_model = torch.load(pretrained_model_path, map_location=torch.device('cpu'))
embedding_model.eval()  # Assuming you're doing inference

# Manually set the embedding_dim if it's not part of the loaded state_dict
embedding_model.embedding_dim = 300

# Manually add the embed method to the loaded model if it's missing
def embed(self, inputs_):
    return self.embeddings(inputs_)

# Assign the method to the instance
embedding_model.embed = embed.__get__(embedding_model, SkipGram_Model)

# Freeze the parameters (weights) of the embedding model
for param in embedding_model.parameters():
    param.requires_grad = False

In [132]:
# Define the path to the combined model
combined_model_path = '/root/MLXWeek2_copy/combined_model.pth'

# Load the combined state dictionary
combined_state = torch.load(combined_model_path, map_location=torch.device('cpu'))

# After loading the state dictionary
print("Keys in the state dict:", combined_state['score_predictor_state_dict'].keys())

# After loading the state dictionary
print("Keys in the state dict:", combined_state['embedding_model_state_dict'].keys())

Keys in the state dict: odict_keys(['embedding_model.embeddings.weight', 'embedding_model.linear.weight', 'embedding_model.linear.bias', 'hidden_1.weight', 'hidden_1.bias', 'hidden_2.weight', 'hidden_2.bias', 'output.weight', 'output.bias'])
Keys in the state dict: odict_keys(['embeddings.weight', 'linear.weight', 'linear.bias'])


In [136]:
score_predictor_model = ScorePredictor(hidden_dim_1, hidden_dim_2, embedding_model)
score_predictor_model.load_state_dict(combined_state['score_predictor_state_dict'])
score_predictor_model.eval()

ScorePredictor(
  (embedding_model): SkipGram_Model(
    (embeddings): Embedding(4000, 300, max_norm=1)
    (linear): Linear(in_features=300, out_features=4000, bias=True)
  )
  (hidden_1): Linear(in_features=300, out_features=64, bias=True)
  (hidden_2): Linear(in_features=64, out_features=32, bias=True)
  (output): Linear(in_features=32, out_features=1, bias=True)
  (relu): ReLU()
)

In [137]:


def score_predictor(title, score_predictor_model):
    # Tokenize the title using the sentencepiece tokenizer
    sp = spm.SentencePieceProcessor(model_file='/root/MLXWeek2_copy/weights/skipgram_TechCrunch/techcrunch_sp.model')
    title_tokens = sp.encode_as_pieces(title)
    title_ids = [sp.piece_to_id(token) for token in title_tokens]

    # Convert to tensor and add batch dimension
    title_tensor = torch.tensor(title_ids).unsqueeze(0)

    # Forward pass through the model
    with torch.no_grad():  # Ensure no gradients are computed
        score = score_predictor_model(title_tensor)

    return score.item()



In [138]:
# Now, use this function to predict the score
title = "The quick brown fox jumps over the lazy dog"
predicted_score = score_predictor(title, score_predictor_model)
print("Predicted score:", predicted_score)

Predicted score: 12.633469581604004
